# ONLY RUN THE NEXT FOUR CELLS ONCE!

## Google Colab Runtime Configuration
**Before running any cells, please change the runtime version:**
1. Go to **Edit** → **Notebook Settings** → **Runtime Version** 
2. Select **2025.07** 
3. Click **Save**

## Important Notes:
- The setup cells below will install dependencies and may cause kernel restarts
- This is expected behavior - simply continue after each restart
- Only run the setup cells once per session

In [ ]:
# Global variable to check if we're in Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    ! pip install numpy==1.26.4
    ### The runtime will crash/restart after this

In [ ]:
# Global variable to check if we're in Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:

  !pip install -q condacolab #-U "git+https://github.com/mayankchetan/condacolab.git@py312update"
  import condacolab
  condacolab.install() #### expect a kernel restart

In [ ]:
# Global variable to check if we're in Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !git clone  https://github.com/wisdem/weis
    !git clone https://github.com/mayankchetan/weisWorkshop

    %cd weis
    !conda config --env --add pinned_packages "numpy<2.0"
    !mamba env update -n base -f environment.yml "python<3.13"
    %pip install .
    %cd ..
    %pip install "rosco==2.9.5"
    %pip install -U "git+https://github.com/NREL/ROSCO@v2.9.7"
    %pip install "orbit-nrel<1.2.2"
    %pip install gdown

else:
    try:
        import weis

    except ImportError:
        raise ImportError("Please install the weis package from https://github.com/wisdem/weis in the environment you are using to run this notebook.")

    Warning("Make sure you have cloned 'https://github.com/mayankchetan/weisWorkshop-Private' repository and are running the notebook from the root directory of the repository.")


In [ ]:
# Downloading the precomputed data from Google Drive & extracting
# Drive link for non Google Colab users: https://drive.google.com/file/d/145BH393nzpT2msCVsHSBxR3Qy8KdpddI/view?usp=sharing
if IN_COLAB:
    !gdown 145BH393nzpT2msCVsHSBxR3Qy8KdpddI -O /content/weisWorkshop/archive_weisWorkshop_OCT25.tar.gz
    !tar -xvzf /content/weisWorkshop/archive_weisWorkshop_OCT25.tar.gz -C /content/weisWorkshop/
else:
    print("Please download the precomputed data from the link given in the notebook and extract it in the root directory of the repository.")
    print("you can use `tar -xvzf <downloaded_file_name>` to extract the files.")

# Welcome to the WEIS Workshop - Chapter 3: Design Load Cases

## Quick Recap

We've successfully developed a complete 20MW floating wind turbine system:
- **Chapter 1**: Optimized rotor design using WISDEM
- **Chapter 2**: Re-tuned controller and optimized platform using RAFT and OpenFAST

## Chapter 3: Design Load Cases (DLC) Analysis

Now we validate our design using industry-standard Design Load Cases (DLCs). This ensures our optimized system meets safety and performance requirements under all operational and extreme conditions.

### What We'll Cover
- **Load Analysis**: Using WEIS to generate a subset of DLCs for our floating wind turbine
- **Running Simulations**: Execute the DLCs using OpenFAST, with WEIS handling the file generation, parallelization, and postprocessing

**Note**: DLC simulations are computationally expensive (~hours per case), so we'll analyze precomputed results that demonstrate the complete validation workflow.

In [ ]:
if IN_COLAB:
    # Now we try loading the weis_workshop module again
    sys.path.append('/content/weisWorkshop/jupyterNotebooks')

# Importing nesscary libraries.
import numpy as np
import matplotlib.pyplot as plt
from weis import weis_main
from wisdem import run_wisdem
import os
import copy
import pandas as pd

from weis_workshop import *

# set env variables
os.environ["OMP_NUM_THREADS"] = "1"  # export OMP_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1"  # export MKL_NUM_THREADS=1

# workshop repository folder
if IN_COLAB:
    WORKSHOP_REPO = '/content/weisWorkshop'
else:
    WORKSHOP_REPO = os.path.join(os.getcwd(), '..')

## Stage 4: Design Load Cases (DLCs)

### Analysis options:

``` yaml
general:
    folder_output: outputs/
    fname_output: stage-4-dlcs
```

### Modeling options (important parts):

``` yaml

OpenFAST: 
    flag: True
    simulation:
    [...]

DLC_driver:
    metocean_conditions:
        wind_speed: [4., 6., 8., 10., 12., 14., 16., 18., 20., 22., 24.]
        wave_height_NSS: [0.83, 0.88, 0.94, 1.03, 1.16, 1.34, 1.57, 1.86, 2.22, 2.62, 3.07]
        wave_period_NSS: [6.9, 6.96, 7.02, 7.12, 7.25, 7.43, 7.66, 7.94, 8.27, 8.63, 9.01]
        wave_height_SSS: [6.3, 8, 8, 8.1, 8.5, 8.5, 9.8, 9.8, 9.8, 9.8, 9.9]
        wave_period_SSS: [11.5, 12.7, 12.7, 12.8, 13.1, 13.1, 14.1, 14.1, 14.1, 14.1, 14.1]
        wave_height_1: 6.98
        wave_period_1: 11.7
        wave_height_50: 10.68
        wave_period_50: 14.2
    DLCs:
        - DLC: "1.1"
          n_seeds: 6
          transient_time: 100.
          analysis_time: 600.
        - DLC: "1.4"
          wind_speed: [9.158, 11.158, 13.158]
        - DLC: "1.6"
          n_seeds: 6
          transient_time: 120.
          analysis_time: 600.
        - DLC: "5.1"
        - DLC: "6.1"
          n_seeds: 6
          transient_time: 120.
          analysis_time: 600.

```

**Key Points:**
- **5 Different DLCs**: Covering normal operation, extreme winds, and extreme waves
- **Realistic Environment**: Wind speeds 4-24 m/s with corresponding wave conditions  
- **Statistical Robustness**: Multiple seeds (6) for turbulent conditions
- **OpenFAST Integration**: Full aeroelastic simulation with platform dynamics

In [ ]:
# Setting up file paths
stageFolder = "stage-4-dlcs"

with open(os.path.join(WORKSHOP_REPO, stageFolder,"outputs_preCompute/dlcs/rank_0/iteration_0/summary_stats.p"), 'rb') as f:
    data = pickle.load(f)

custom_y_params = [
            ('GenSpeed', 'mean'),
            ('RootMyb1', 'max'),
            ('TwrBsAxMxyt', 'median'),
            ('PtfmPitch', 'max')
]

plot_weis_summary(data, custom_y_params)

In [ ]:
custom_y_params = [
            ('RootMxb1', 'max'),
            ('RootMyb1', 'max'),
            ('TwrBsMxt', 'median'),
            ('TwrBsMyt', 'median'),
]

plot_weis_summary(data, custom_y_params)

# Workshop Summary

## What We've Accomplished

This workshop has taken you through the complete **multi-fidelity co-design workflow** used in modern floating offshore wind turbine development. You've experienced the same systematic approach used by industry and research organizations worldwide.

### Chapter 1: Foundation - WISDEM Rotor Optimization
- **Started** with the IEA 22MW baseline floating wind turbine
- **Scaled** the design to create a 20MW turbine with 270m rotor diameter  
- **Optimized** rotor aerodynamics and structure using steady-state WISDEM models
- **Learned** about design variables, constraints, and merit figures in wind turbine optimization

### Chapter 2: Integration - Multi-Fidelity System Co-Design
- **Re-tuned** the ROSCO controller for our new rotor design using OpenFAST
- **Optimized** the floating platform using RAFT frequency-domain analysis
- **Refined** the platform design with high-fidelity OpenFAST time-domain simulations
- **Experienced** the systematic progression from fast models to detailed analysis

### Chapter 3: Validation - Design Load Cases Analysis
- **Applied** industry-standard IEC 61400-3 Design Load Cases
- **Validated** our optimized system against operational and extreme conditions
- **Analyzed** comprehensive load statistics across multiple environmental scenarios
- **Confirmed** our design meets safety and performance requirements

## Key WEIS Capabilities Demonstrated

**Multi-Tool Integration:**
- WISDEM for system engineering and initial design
- RAFT for rapid floating platform analysis  
- OpenFAST for detailed aeroelastic simulation
- ROSCO for advanced controller optimization

**Multi-Fidelity Methodology:**
- Strategic use of appropriate model fidelity for each design phase
- Efficient progression from conceptual design to detailed validation
- Balanced computational cost with engineering accuracy

**Industry-Standard Workflows:**
- Design Load Case generation and analysis
- Optimization under realistic constraints
- Comprehensive load assessment and validation

## Next Steps

To continue developing your WEIS skills:
1. **Explore the examples** in the WEIS repository for different turbine configurations
2. **Modify design variables** and constraints to understand their impacts
3. **Run additional DLCs** for comprehensive design validation
4. **Experiment with different optimization algorithms** and convergence criteria

The WEIS framework provides the foundation for advanced floating offshore wind research and development. The skills you've developed here prepare you to tackle real-world engineering challenges in this rapidly growing field.